In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Models

##Библиотеки

In [2]:
! pip install pytelegrambotapi -q
! pip install SpeechRecognition pydub -q
!pip install deepface -q
! pip install speechbrain -q
! pip install onnxruntime -q
! pip install pytube -q
import cv2
import sys
import numpy as np
import telebot
from telebot import types
import os
import speech_recognition as sr
import uuid
from PIL import Image
import requests
from io import BytesIO
from torch.utils import model_zoo
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
%pylab inline
from deepface import DeepFace
from tqdm import tqdm
import pandas as pd
from speechbrain.pretrained import EncoderClassifier
import onnxruntime
import torchaudio
import json
from pytube.exceptions import VideoUnavailable
from pytube import YouTube
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.9/222.9 KB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 299.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.0/499.0 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

##LooksLike_______task_type = 1

In [3]:
def LooksLike(path_t):
  def verify_func(characters, target_img_name):
      min_dist = float('inf')
      path = None
      for character in tqdm(characters):
          obj = DeepFace.verify(img1_path = character, img2_path = target_img_name, 
                              model_name = 'Facenet', enforce_detection = False)
          if obj['distance'] < min_dist:
              min_dist = obj['distance']
              path = character
      return path

  target_img_name = path_t
  characters = []
  for dirpath, dirnames, filenames in os.walk('/content/gdrive/MyDrive/Celebrity_Faces_Dataset_small'):
      for filename in filenames:
          if '.jpg' in filename:
              characters.append(dirpath + '/' + filename)
  img_name = verify_func(characters, target_img_name)  
  return Image.open(img_name)

## Speech->Text_________task_type = 2

In [4]:
def speech(filename):
  r = sr.Recognizer() 
  with sr.AudioFile(filename) as source:
      audio_text = r.listen(source)
      try:
          text = r.recognize_google(audio_text, language="ru-RU")
          return text
      except:
          return "Попробуйте ещё раз." 

## 3D Model_____task_type = 3

In [5]:
def DDD_model(file_im):
    cd_1=['0', '0', '0']# создание массива для хранения координат 1 вершины
    cd_2=['0', '0', '0']# создание массива для хранения координат 2 вершины
    cd_3=['0', '0', '0']# создание массива для хранения координат 3 вершины

    file_stl = 'model.stl'
    op_stl=open(file_stl, 'w')
    op_im=cv2.imread(file_im)
    gray = cv2.cvtColor(op_im, cv2.COLOR_BGR2GRAY)#преобразование в чб изображение
    blur = cv2.GaussianBlur(gray,(0,0),1)# небольшое размытие для сглажеввания шумов
    blur=cv2.resize(blur,(320,240))
    x=0
    y=0
    file='STL_project-1.stl'
    o_1="\n\t"
    o_2="\n\t\t"
    o_3="\n\t\t\t"
    op_stl.write("solid")

    def face_file_stl(cd_1, cd_2, cd_3):
      op_stl.write(o_1+"facet normal 0 0 0")
      op_stl.write(o_2 + "outer loop")
      op_stl.write(o_3 + "vertex " + " ".join(cd_1))#запись в файл координаты 1 вершины
      op_stl.write(o_3 + "vertex " + " ".join(cd_2))#запись в файл координаты 2 вершины
      op_stl.write(o_3 + "vertex " + " ".join(cd_3))#запись в файл координаты 3 вершины
      op_stl.write(o_2 + "endloop \n\tendfacet")

    #making the first triangls  for base
    for i in range(blur.shape[1]-1):
          cd_1=[str(i),"0","0"]
          cd_3=[str(i+1),str(blur.shape[0]-1),"0"]
          cd_2=[str(i),str(blur.shape[0]-1),"0"]
          face_file_stl(cd_1, cd_2, cd_3)
    #making the second triangls  for base      
    for i in range(blur.shape[1]-1):
          cd_1=[str(i+1),str(blur.shape[0]-1),"0"]
          cd_3=[str(i),"0","0"]
          cd_2=[str(i+1),"0","0"]
          face_file_stl(cd_1, cd_2, cd_3)
    #base has done

    for i in range(blur.shape[1]):
             
            for k in range(blur.shape[0]-1):#making the first triangls  for relief
                if i!=blur.shape[1]-1:
                        try:
                            
                            cd_1=[str(i),     str(k),   str(blur[k, i])   ]
                            cd_2=[str(i + 1), str(k),   str(blur[k, i+1]) ]
                            cd_3=[str(i+1),   str(k+1), str(blur[k+1,i+1])] 
                        
                        except:
                            print('er')
                        face_file_stl(cd_1, cd_2, cd_3)
                  #for j in range(blur.shape[1]-1):#making the second triangls  for relief
                        try:
                            cd_1=[str(i),  str(k),   str(blur[k, i])    ]
                            cd_2=[str(i+1),str(k+1), str(blur[k+1, i+1])]
                            cd_3=[str(i),  str(k+1), str(blur[k+1,i])   ]   
                        except:
                            print('er')
                        face_file_stl(cd_1, cd_2, cd_3)
    #relief has done

    #making the first triangls  for right side        
    for i in range(blur.shape[1]):
        if i!=blur.shape[1]-1:
          try:
                cd_1=[str(i),str(blur.shape[0]-1),"0"]
                cd_3=[str(i+1),str(blur.shape[0]-1),str(blur[blur.shape[0]-1, i+1])]
                cd_2=[str(i),str(blur.shape[0]-1),str(blur[blur.shape[0]-1, i])]
                
          except:
                print("er")
          face_file_stl(cd_1, cd_2, cd_3)
          
    #making the second triangls  for right side     
          try:
                cd_1=[str(i),str(blur.shape[0]-1),"0"]
                cd_3=[str(i+1),str(blur.shape[0]-1),"0"]
                cd_2=[str(i+1),str(blur.shape[0]-1),str(blur[blur.shape[0]-1, i+1])]
          except:
                print("er")
          face_file_stl(cd_1, cd_2, cd_3)

        if i!=blur.shape[1]-1:
          try:
                cd_1=[str(i),'0',"0"]
                cd_2=[str(i+1),'0',str(blur[0, i+1])]
                cd_3=[str(i),'0',str(blur[0, i])]
                
          except:
                print("er")
          face_file_stl(cd_1, cd_2, cd_3)
          
    #making the second triangls  for right side     
          try:
                cd_1=[str(i),'0',"0"]
                cd_2=[str(i+1),'0',"0"]
                cd_3=[str(i+1),'0',str(blur[0, i+1])]
          except:
                print("er")
          face_file_stl(cd_1, cd_2, cd_3)

    for i in range(blur.shape[0]):
      if i!=blur.shape[0]-1:
          try:
                cd_1=['0',str(i),"0"]
                cd_3=['0',str(i+1),str(blur[ i+1,0])]
                cd_2=['0',str(i),str(blur[i,0])]
                
          except:
                print("er")
          face_file_stl(cd_1, cd_2, cd_3)
          
    #making the second triangls  for right side     
          try:
                cd_1=['0',str(i),"0"]
                cd_3=['0',str(i+1),"0"]
                cd_2=['0',str(i+1),str(blur[i+1,0])]
          except:
                print("er")
          face_file_stl(cd_1, cd_2, cd_3)
      
          try:
                cd_2=[str(blur.shape[1]-1),str(i),"0"]
                cd_3=[str(blur.shape[1]-1),str(i+1),str(blur[ i+1,blur.shape[1]-1])]
                cd_1=[str(blur.shape[1]-1),str(i),str(blur[i,blur.shape[1]-1])]
              
          except:
                print("er")
          face_file_stl(cd_1, cd_2, cd_3)
          
    #making the second triangls  for right side     
          try:
                cd_2=[str(blur.shape[1]-1),str(i),"0"]
                cd_3=[str(blur.shape[1]-1),str(i+1),"0"]
                cd_1=[str(blur.shape[1]-1),str(i+1),str(blur[i+1,blur.shape[1]-1])]
          except:
                print("er")

          face_file_stl(cd_1, cd_2, cd_3)

    op_stl.write("\nendsolid" )
    op_stl.close()
    return file_stl

## Skleyka_______task_type = 4

In [6]:
# task_type = 4
def Skleyka(im1, im2):
  img1 = cv2.imread(im1)
  img2 = cv2.imread(im2)

  img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
  img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

  orb = cv2.ORB_create(nfeatures=2000)
  keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
  keypoints2, descriptors2 = orb.detectAndCompute(img2, None)
  bf = cv2.BFMatcher_create(cv2.NORM_HAMMING)
  matches = bf.knnMatch(descriptors1, descriptors2, k = 2)

  def draw_matches(img1, keypoints1, img2, keypoints2, matches):
    r, c = img1.shape[:2]
    r1, c1 = img2.shape[:2]

    output_img = np.zeros((max([r, r1]), c+c1, 3), dtype='uint8')
    output_img[:r, :c, :] = np.dstack([img1, img1, img1])
    output_img[:r1, c:c+c1, :] = np.dstack([img2, img2, img2])

    for match in matches:
      img1_idx = match.queryIdx
      img2_idx = match.trainIdx
      (x1, y1) = keypoints1[img1_idx].pt
      (x2, y2) = keypoints2[img2_idx].pt
      cv2.circle(output_img, (int(x1),int(y1)), 4, (0, 255, 255), 1)
      cv2.circle(output_img, (int(x2)+c,int(y2)), 4, (0, 255, 255), 1)
      cv2.line(output_img, (int(x1),int(y1)), (int(x2)+c,int(y2)), (0, 255, 255), 1)
    return output_img
  all_matches = []
  for m, n in matches:
    all_matches.append(m)

  img3 = draw_matches(img1_gray, keypoints1, img2_gray, keypoints2, all_matches[:30])
  good = []
  for m, n in matches:
      if m.distance < 0.95 * n.distance:
          good.append(m)
  def warpImages(img1, img2, H):

    rows1, cols1 = img1.shape[:2]
    rows2, cols2 = img2.shape[:2]

    list_of_points_1 = np.float32([[0,0], [0, rows1],[cols1, rows1], [cols1, 0]]).reshape(-1, 1, 2)
    temp_points = np.float32([[0,0], [0,rows2], [cols2,rows2], [cols2,0]]).reshape(-1,1,2)
    list_of_points_2 = cv2.perspectiveTransform(temp_points, H)

    list_of_points = np.concatenate((list_of_points_1,list_of_points_2), axis=0)

    [x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() + 0.5)
    
    translation_dist = [-x_min,-y_min]
    
    H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])

    output_img = cv2.warpPerspective(img2, H_translation.dot(H), (x_max-x_min, y_max-y_min))
    output_img[translation_dist[1]:rows1+translation_dist[1], translation_dist[0]:cols1+translation_dist[0]] = img1

    return output_img

  MIN_MATCH_COUNT = 10

  if len(good) > MIN_MATCH_COUNT:
      # Convert keypoints to an argument for findHomography
      src_pts = np.float32([ keypoints1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
      dst_pts = np.float32([ keypoints2[m.trainIdx].pt for m in good]).reshape(-1,1,2)

      # Establish a homography
      M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
      
      result = warpImages(img2, img1, M)
      cv2.imwrite('result.jpg', result) 

## PhotoDescriber________task_type = 5

In [7]:
params = model_zoo.load_url('https://s3.amazonaws.com/modelzoo-networks/nin-export-e65df26d.pth')
for k,v in sorted(params.items()):
    params[k] = Variable(v)

def network_in_network(inputs, params):
    def conv2d(x, params, name, stride=1, padding=0):
        return F.conv2d(x,
                        params['%s.weight'%name],
                        params['%s.bias'%name], stride, padding)

    def block(x, names, stride, padding):
        x = F.relu(conv2d(x, params, names[0], stride, padding))
        x = F.relu(conv2d(x, params, names[1]))
        x = F.relu(conv2d(x, params, names[2]))
        return x

    o = block(inputs, ['conv0', 'conv1', 'conv2'], 4, 5)
    o = F.max_pool2d(o, 2)
    o = block(o, ['conv3', 'conv4', 'conv5'], 1, 2)
    o = F.max_pool2d(o, 2)
    o = block(o, ['conv6', 'conv7', 'conv8'], 1, 1)
    o = F.max_pool2d(o, 2)
    o = block(o, ['conv9', 'conv10', 'conv11'], 1, 1)
    o = F.avg_pool2d(o, 7)
    o = o.view(o.size(0), -1)
    o = F.linear(o, params['fc.weight'], params['fc.bias'])
    return o

def things_classification(path):

  def load(url):
      #response = requests.get(url)
      response = cv2.imread(url)
      #return np.ascontiguousarray(Image.open(BytesIO(response.content)), dtype=np.uint8)
      response = cv2.cvtColor(response, cv2.COLOR_RGB2BGR)

      return np.ascontiguousarray(response, dtype=np.uint8)

  im = load(path)
  tr_center_crop = transforms.Compose([
          transforms.ToPILImage(),
          transforms.Resize(256),
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ])

  def classify(x):
      with open('/content/gdrive/MyDrive/synset_words.txt') as h:
          synset_words = [s[10:-1] for s in h.readlines()]

      with torch.no_grad():
          y = network_in_network(Variable(x.unsqueeze(0)), params)
          probs, idx = F.softmax(y, dim=1).data.squeeze(0).sort(0,True)

      text = '\n'.join('%.2f: %s' % (p, synset_words[class_id])
                      for p, class_id in zip(probs[:5], idx[:5]))
      return text

  return (classify(tr_center_crop(im)))

Downloading: "https://s3.amazonaws.com/modelzoo-networks/nin-export-e65df26d.pth" to /root/.cache/torch/hub/checkpoints/nin-export-e65df26d.pth


  0%|          | 0.00/33.0M [00:00<?, ?B/s]

## CelebrityVoice_____task_type = 6

In [8]:
def audio_to_features(classifier, signal, signal_len, device='cpu'):
    """
    """
    wavs = signal
    wav_lens = signal_len

    # Manage single waveforms in input
    if len(wavs.shape) == 1:
        wavs = wavs.unsqueeze(0)
    # Assign full length if wav_lens is not assigned
    if wav_lens is None:
        wav_lens = torch.ones(wavs.shape[0], device=device)
    # Storing waveform in the specified device
    wavs, wav_lens = wavs.to(device), wav_lens.to(device)
    wavs = wavs.float()
    # Computing features and embeddings
    feats = classifier.mods.compute_features(wavs)
    feats = classifier.mods.mean_var_norm(feats, wav_lens)

    # for tg bot
    feats = feats.cpu().numpy()

    return feats, wav_lens


def classify_batch_speechbrain(classifier, sound_embedding, emb_lens):
    voice_embedding = classifier.encode_batch(sound_embedding, emb_lens)
    class_probabilities = classifier.mods.classifier(voice_embedding).squeeze(1)
    return class_probabilities


def post_processing(labels, class_scores, top_k=2):
    values, indices = torch.topk(class_scores, top_k, dim=-1)
    top_results = []
    for i in range(top_k):
        index = indices[0][i].item()
        label_id = labels[str(index)]
        top_results.append((label_id, values[0][i].item()))

    return top_results


def build_classifier_speechbrain(device='cpu'):
    classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb",
                                                savedir="pretrained_models/spkrec-xvect-voxceleb",
                                                run_opts={"device": device})
    classifier.eval()
    return classifier


def build_label_decoder(labels_path='/content/gdrive/MyDrive/project/vox_ids.json'):
    with open(labels_path, 'r') as fp:
        labels = json.load(fp)
    return labels


def get_photo_coords(label_id):
    df = pd.read_csv('/content/gdrive/MyDrive/project/all_vox.csv', sep='\t')
    reference = df.loc[df['id'] == label_id, ['reference']].values[0][0]
    frame = df.loc[df['id'] == label_id, ['frame']].values[0][0]

    return str(reference), int(frame)


def get_name(label_id):
     

    # Получить объект YouTube видео по URL
    reference = get_photo_coords(label_id)[0]
    try:
        video = YouTube(f"https://www.youtube.com/watch?v={reference}")
    except VideoUnavailable:
        print("Видео недоступно.")
    title = video.title

    interviewee_name = re.search(r'Interview(er|ee|ee:|ed by| with|s|ed)\s?(.+?)( about| on| in|,|:|$)', title)
    if interviewee_name:
        print(interviewee_name.group(2))

def get_frame(reference, frame_number):

    try:
        yt = YouTube(f"https://www.youtube.com/watch?v={reference}")
        stream = yt.streams.get_highest_resolution()
        stream.download(filename='video.mp4')
    except VideoUnavailable:
        return None

    cap = cv2.VideoCapture('video.mp4')
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    os.remove('video.mp4')
    return frame

def crop_image(image, id, i):
    from PIL import Image

    all_vox = pd.read_csv('./all_vox.csv', sep='\t')
    image = Image.open(image)

    x = all_vox[all_vox.id == id].x.iloc[0]
    y = all_vox[all_vox.id == id].y.iloc[0]
    w = all_vox[all_vox.id == id].w.iloc[0]
    h = all_vox[all_vox.id == id].h.iloc[0]
    if w < 1.:
        x, w = x * image.size[0], w * image.size[0]
        y, h = y * image.size[0], h * image.size[0]
    im_crop = image.crop((x, y, x + w, y + h))
    im_crop.save(f'celeb_image_{i}.jpg', quality=95)

def get_signal(filename): 
    signal, _ = torchaudio.load(filename)
    return signal


def main(classifier, signal, label_decoder, signal_len=None, top_k=2):
    signal.clone().detach()

    emb, emb_len = audio_to_features(classifier, signal, signal_len)

    class_probs = classify_batch_speechbrain(classifier, signal, emb_len)

    top_results = post_processing(label_decoder, class_probs, top_k=top_k)

    return top_results, emb


def CelebrityVoice(path_voice):
    signal = get_signal(path_voice)
    label_decoder = build_label_decoder()
    sp_classifier = build_classifier_speechbrain()
    top_results, _ = main(sp_classifier, signal, label_decoder, top_k=2)

    for i, (label_id, score) in enumerate(top_results):        
        try:
            reference, frame_num = get_photo_coords(label_id)
            frame = get_frame(reference, frame_num)
            if frame is not None and not frame.size == 0:
                cv2.imwrite(f'frame_{i}.jpg', frame)
                crop_image(f'frame_{i}.jpg', label_id, i)
                get_name(label_id)
                print(f'Done for result {i}!')
                break
            else:
                raise Exception(f"Empty or invalid frame for result {i}.")
        except (VideoUnavailable, Exception) as e:
            if i == len(top_results) - 1:
                print("All top results exhausted.")

# Bot

In [ ]:
bot = telebot.TeleBot('6134902648:AAFDPQBp2ZoEXuhntTnAJgexVIBmlY_WqZo');

@bot.message_handler(commands=['start'])
def start(message):
  bot.send_message(message.chat.id,f"Привет, {message.from_user.first_name}!👋")
  markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width = 2)
  btn1 = types.KeyboardButton("LooksLike")
  btn2 = types.KeyboardButton('Speech->Text')
  btn3 = types.KeyboardButton('3D Model')
  btn4 = types.KeyboardButton('Skleyka')
  btn5 = types.KeyboardButton('PhotoDescriber')
  btn6 = types.KeyboardButton('CelebrityVoice') # не work

  bot.send_message(message.from_user.id, "Спасибо за разработку (или просто готовый код):\nLooksLike - @sslatyshev\nSpeech->Text - @EnderPortman\n3D Model - @cloud_01_24\nSkleyka - @cloud_01_24\nPhotoDescriber - @cloud_01_24\nCelebrityVoice - @nkl_c & @elizzz13")

  markup.add(btn1, btn2, btn3, btn4, btn5, btn6)
  bot.send_message(message.from_user.id, "Выбери действие🔽", reply_markup=markup)


task_type = 0
@bot.message_handler(content_types=['text'])
def text(message):
  global task_type
  mess = message.text

  if mess == "LooksLike":
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
    btn = types.KeyboardButton("Хочу выбрать снова↩️")
    markup.add(btn)
    task_type = 1
    ans = "Жду твое фото😊📷"
    bot.send_message(message.from_user.id, ans, reply_markup=markup)
  
  elif mess == "Speech->Text":
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
    btn = types.KeyboardButton("Хочу выбрать снова↩️")
    markup.add(btn)
    task_type = 2
    ans = "Жду аудио🎧"
    bot.send_message(message.from_user.id, ans, reply_markup=markup)
    bot.send_message(message.from_user.id, "Гс не более 15 сек на русском\nФункция может сработать не сразу, так как это бесплатно :)")

  elif mess == "3D Model":
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
    btn = types.KeyboardButton("Хочу выбрать снова↩️")
    markup.add(btn)
    task_type = 3
    ans = "Жду картинку🌇"
    bot.send_message(message.from_user.id, ans, reply_markup=markup)

  elif mess == "Skleyka":
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
      btn = types.KeyboardButton("Хочу выбрать снова↩️")
      markup.add(btn)
      task_type = 4
      ans = "Жду два фото с разного ракурса🌇🌆"
      bot.send_message(message.from_user.id, ans, reply_markup=markup)
  
  elif mess == "PhotoDescriber":
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
    btn = types.KeyboardButton("Хочу выбрать снова↩️")
    markup.add(btn)
    task_type = 5
    ans = "Жду картинку зверя🐸 или чего угодно🎃"
    bot.send_message(message.from_user.id, ans, reply_markup=markup)

  elif mess == "CelebrityVoice":
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
    btn = types.KeyboardButton("Хочу выбрать снова↩️")
    markup.add(btn)
    task_type = 6
    ans = "Скажи мне что-нибудь🗣"
    bot.send_message(message.from_user.id, ans, reply_markup=markup)

  elif mess == "Хочу выбрать снова↩️" or mess == "Круто! Давай ещё🤩" or mess == "👌" or mess == "Давай!":
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
    btn1 = types.KeyboardButton("LooksLike")
    btn2 = types.KeyboardButton('Speech->Text')
    btn3 = types.KeyboardButton('3D Model')
    btn4 = types.KeyboardButton('Skleyka')
    btn5 = types.KeyboardButton('PhotoDescriber')
    btn6 = types.KeyboardButton('CelebrityVoice')

    markup.add(btn1, btn2, btn3, btn4, btn5, btn6)
    ans = "Выбери действие🔽"
    bot.send_message(message.from_user.id, ans, reply_markup=markup) 

  else:
    print(task_type)
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
    btn = types.KeyboardButton("Хочу выбрать снова↩️")
    task_type = 0
    markup.add(btn)
    bot.send_message(message.from_user.id, text = "🐈", reply_markup=markup)    

count_files = 0
@bot.message_handler(content_types=['photo'])
def photo(message):
  global task_type
  global count_files
  if task_type in [1, 3, 4, 5]:
    count_files += 1

    if count_files == 2 and task_type == 4:
      photo_id = message.photo[-1].file_id
      photo_file = bot.get_file(photo_id)
      photo_bytes = bot.download_file(photo_file.file_path)
      with open("im2.jpg", 'wb') as new_file:
            new_file.write(photo_bytes)

      Skleyka("im1.jpg", "im2.jpg")
      photo = open('result.jpg', 'rb')
      bot.send_photo(message.chat.id, photo)
      
      ans = 'И как?🧐'
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
      btn = types.KeyboardButton("Круто! Давай ещё🤩")
      markup.add(btn)
      bot.send_message(message.from_user.id, ans, reply_markup=markup)
      task_type = 0
      count_files = 0

    elif count_files == 1:
      photo_id = message.photo[-1].file_id
      photo_file = bot.get_file(photo_id) 
      photo_bytes = bot.download_file(photo_file.file_path)
      with open("im1.jpg", 'wb') as new_file:
            new_file.write(photo_bytes)

      if task_type == 1:
        bot.send_message(message.from_user.id, 'Я тебя узнал!\nМне нужно 8 минут, чтобы найти фото...')
        bot.send_message(message.from_user.id, '👀')
        im1 = LooksLike("im1.jpg")
        bot.send_photo(message.chat.id, im1)
        ans = 'И как?🧐'
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
        btn = types.KeyboardButton("Круто! Давай ещё🤩")
        markup.add(btn)
        bot.send_message(message.from_user.id, ans, reply_markup=markup)
        task_type = 0
        count_files = 0

      elif task_type == 3:
        DDD_model("im1.jpg")
        doc = open('/content/model.stl')
        bot.send_document(message.chat.id, doc)
        ans = 'И как?🧐'
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
        btn = types.KeyboardButton("Круто! Давай ещё🤩")
        markup.add(btn)
        bot.send_message(message.from_user.id, ans, reply_markup=markup)
        task_type = 0
        count_files = 0
      
      elif task_type == 5:
        text = things_classification("im1.jpg")
        bot.send_message(message.from_user.id, 'Вероятности c описанием готовы!🥳')
        bot.send_message(message.from_user.id, text)
        ans = 'И как?🧐'
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
        btn = types.KeyboardButton("Круто! Давай ещё🤩")
        markup.add(btn)
        bot.send_message(message.from_user.id, ans, reply_markup=markup)
        task_type = 0
        count_files = 0      

  else:
    task_type = 0
    count_files = 0
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
    btn = types.KeyboardButton("Хочу выбрать снова↩️")
    markup.add(btn)
    bot.send_message(message.from_user.id, text = "🐈", reply_markup=markup)

@bot.message_handler(content_types=['voice'])
def voice_rec(message):
  global task_type
  if task_type in [2, 6]:
    if task_type == 2:
      bot.send_message(message.from_user.id, "Какой красивый голос🥰")
      filename = str(uuid.uuid4())
      file_name_full=filename+".ogg"
      file_name_full_converted=filename+".wav"
      file_info = bot.get_file(message.voice.file_id)
      downloaded_file = bot.download_file(file_info.file_path)
      with open(file_name_full, 'wb') as new_file:
          new_file.write(downloaded_file)
      os.system("ffmpeg -i "+file_name_full+"  "+file_name_full_converted)
      text=speech(file_name_full_converted)

      bot.send_message(message.from_user.id, text = text)
      ans = 'Хочешь ещё?🤔'
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
      btn = types.KeyboardButton("Давай!")
      markup.add(btn)
      bot.send_message(message.from_user.id, ans, reply_markup=markup)
      task_type = 0 

    elif task_type == 6:
      bot.send_message(message.from_user.id, "Приятный голос🤤")
      filename = str(uuid.uuid4())
      file_name_full=filename+".ogg"
      file_name_full_converted=filename+".wav"
      file_info = bot.get_file(message.voice.file_id)
      downloaded_file = bot.download_file(file_info.file_path)
      with open(file_name_full, 'wb') as new_file:
          new_file.write(downloaded_file)
      os.system("ffmpeg -i "+file_name_full+"  "+file_name_full_converted)

      CelebrityVoice(file_name_full_converted)

      bot.send_message(message.from_user.id, "У тебя такой же голос, как у....")
      photo = open('frame_0.jpg', 'rb')
      bot.send_photo(message.chat.id, photo)

      ans = 'Хочешь ещё?🤔'
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
      btn = types.KeyboardButton("Давай!")
      markup.add(btn)
      bot.send_message(message.from_user.id, ans, reply_markup=markup)
      task_type = 0 

  else:
    task_type = 0
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
    btn = types.KeyboardButton("Хочу выбрать снова↩️")
    markup.add(btn)
    bot.send_message(message.from_user.id, text = "🐈", reply_markup=markup)  
  

bot.polling(none_stop=True, interval=0)

All top results exhausted.
